In [14]:
import pandas as pd
import re
import requests
import numpy as np
from bs4 import BeautifulSoup
from glob import iglob

state_df = pd.read_excel(open(next(iglob('Community_Profile_Report*.xlsx')),'rb'), sheet_name='States')

# make first row as header
state_df.columns = state_df.iloc[0]
state_df = state_df[1:]

state_df = state_df.rename(columns={"People with at least 1 dose as % of total population": "People initiating vaccination as % of total population", 
                   "People who are fully vaccinated as % of total population": "People with full course administered as % of total population"})

state_df = state_df[['State', 'State Abbreviation', 'Doses distributed', 'Doses distributed per 100k population',
       'Doses administered', 'Doses administered per 100k population',
       'People initiating vaccination as % of total population',
       'People with full course administered as % of total population',
         'Cumulative cases', 'Cumulative deaths',
         'Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting)']]



state_df.to_csv("cdcCovidDataByState.csv", index=False)
# state_df

In [15]:
df = pd.read_excel(open(next(iglob('Community_Profile_Report*.xlsx')),'rb'), sheet_name='Counties')

# make first row as header
df.columns = df.iloc[0]
df = df[1:]

# drop unallocated columns
df=df.drop(df[df.County.str.contains("Unallocated")].index)

# add zero to 4-digit fips and convert to string for GeoJson join
df["FIPS code"] = df["FIPS code"].astype(str).str.zfill(5)

df=df.rename(columns={"FIPS code" : "county_fips"})

df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('SustainedHotspot', 'Sustained Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('Hotspot', 'Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('HighBurdenResolving', 'High Burden Resolving')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('ModerateBurdenResolving', 'Moderate Burden Resolving')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('EmergingHotspot', 'Emerging Hotspot')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('ModerateBurden', 'Moderate Burden')
df['Area of Concern Category'] = df['Area of Concern Category'].str.replace('LowBurden', 'Low Burden')

df.head()

df[['County', 'county_fips','Cases per 100k - last 7 days', 'Cases - last 7 days', 'Deaths - last 7 days',
 'Deaths per 100k - last 7 days', 'Cases - % change', 'Deaths - % change', 'Cumulative cases','Cumulative deaths',
 'Number of days of downward case trajectory', 'Area of Concern Category', 'Rapid Riser Category', 
'Viral (RT-PCR) lab test positivity rate - last 7 days (may be an underestimate due to delayed reporting)',
 'Confirmed COVID-19 admissions - last 7 days', 'RT-PCR tests per 100k - last 7 days (may be an underestimate due to delayed reporting)',
 'Area of Concern Category', '% inpatient beds occupied', '% staffed adult ICU beds occupied', 
 '% inpatient beds occupied by COVID-19 patient', '% staffed adult ICU beds occupied by COVID-19 patient'
#save the file to be uploaded to github
# df.to_csv("latest-cdc-covid-data-by-county.csv", index=False)

,County,county_fips,County type,CBSA,CBSA type,State Abbreviation,FEMA region,Population,Population as a percent of CBSA,Population as a percent of state,...,% In Poverty,% Over Age 65,Average household size,% Non-Hispanic Black,% Hispanic,% Native American / Alaskan Native,% Asian,SVI score,CCVI score,Forecasted case trajectory
1,"Los Angeles County, CA",06037,Large central metro,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan,CA,Region 9,10039107,0.7597,0.2541,...,0.159872,0.140825,3,0.0795169,0.486295,0.01,0.153942,0.7682,0.489335,Uncertain
2,"Kings County, NY",36047,Large central metro,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan,NY,Region 2,2559903,0.1332,0.1316,...,0.211454,0.14371,2.69,0.297131,0.188944,0.01,0.126641,0.8153,0.912448,Likely Decreasing
3,"Miami-Dade County, FL",12086,Large central metro,"Miami-Fort Lauderdale-Pompano Beach, FL",Metropolitan,FL,Region 4,2716940,0.4406,0.1265,...,0.180398,0.166587,3.07,0.153529,0.694297,0,0.0162355,0.8175,0.81439,Uncertain
4,"Queens County, NY",36081,Large central metro,"New York-Newark-Jersey City, NY-NJ-PA",Metropolitan,NY,Region 2,2253858,0.1173,0.1159,...,0.129926,0.163423,2.91,0.179261,0.28171,0.01,0.269031,0.6201,0.832219,Likely Decreasing
5,"Harris County, TX",48201,Large central metro,"Houston-The Woodlands-Sugar Land, TX",Metropolitan,TX,Region 6,4713325,0.667,0.1626,...,0.162248,0.109088,2.88,0.188181,0.437275,0.01,0.0732829,0.7236,0.557147,Uncertain


In [12]:
state_df.columns

Index(['State', 'State Abbreviation', 'FEMA region', 'Population',
       'Population as a percent of national population',
       'Cases as a percent of national total - last 7 days',
       'Cases - last 7 days', 'Cases per 100k - last 7 days',
       'Deaths - last 7 days', 'Deaths per 100k - last 7 days',
       ...
       'Flu vaccination rate upper limit (2018-2019) - Ages 5-12',
       'Flu vaccination rate lower limit (2018-2019) - Ages 13-17',
       'Flu vaccination rate (2018-2019) - Ages 13-17',
       'Flu vaccination rate upper limit (2018-2019) - Ages 13-17',
       'Flu vaccination rate lower limit (2018-2019) - Ages 18-64',
       'Flu vaccination rate (2018-2019) - Ages 18-64',
       'Flu vaccination rate upper limit (2018-2019) - Ages 18-64',
       'Flu vaccination rate lower limit (2018-2019) - Ages 65+',
       'Flu vaccination rate (2018-2019) - Ages 65+',
       'Flu vaccination rate upper limit (2018-2019) - Ages 65+'],
      dtype='object', name=0, length=136